In [2]:
# import cv2 
import argparse

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

import datasets
import util.misc as utils
from models import build_model, build_model_hierINTR
import datasets.transforms as T
from datasets import build_dataset

In [3]:
def get_args_parser():
    parser = argparse.ArgumentParser('Set transformer detector', add_help=False)
    parser.add_argument('--lr_backbone', default=1.00e-5, type=float)
    parser.add_argument('--batch_size', default=1, type=int, choices=[1])

    # * Backbone
    parser.add_argument('--backbone', default='resnet50', type=str,
                        help="Name of the convolutional backbone to use")
    parser.add_argument('--dilation', action='store_true',
                        help="If true, we replace stride with dilation in the last convolutional block (DC5)")
    parser.add_argument('--position_embedding', default='sine', type=str, choices=('sine', 'learned'),
                        help="Type of positional embedding to use on top of the image features")
    parser.add_argument('--resume', default='path/to/intr_checkpoint_cub_detr_r50.pth',
                        help='resume from checkpoint')
    
    # * Transformer
    parser.add_argument('--enc_layers', default=6, type=int,
                        help="Number of encoding layers in the transformer")
    parser.add_argument('--dec_layers', default=6, type=int,
                        help="Number of decoding layers in the transformer")
    parser.add_argument('--dim_feedforward', default=2048, type=int,
                        help="Intermediate size of the feedforward layers in the transformer blocks")
    parser.add_argument('--hidden_dim', default=256, type=int,
                        help="Size of the embeddings (dimension of the transformer)")
    parser.add_argument('--dropout', default=0.1, type=float,
                        help="Dropout applied in the transformer") #default=0.1
    parser.add_argument('--nheads', default=8, type=int,
                        help="Number of attention heads inside the transformer's attentions")
    parser.add_argument('--num_queries', default=200, type=int,
                        help="Number of query slots")
    parser.add_argument('--pre_norm', action='store_true')

    # # * Dataset parameters
    parser.add_argument('--dataset_name', default='cub') 
    parser.add_argument('--dataset_path', default='/path/to/datasets', type=str) 
    parser.add_argument('--test', default="val", type=str, choices=["val", "test"])

    # * Device parameters
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')

    # Phylogeny parameters
    parser.add_argument("--phylo_config", type=str, default=None, 
                        help='path to the yaml file containing "phylogeny_path" and "phyloDistances_string"') # "./configs/cub27_phylogeny.yaml"
    
    return parser

In [4]:
data_mean_std = {
    "cub": (
        torch.tensor([0.485, 0.456, 0.406]),
        torch.tensor([0.229, 0.224, 0.225]),
    ),
}

class SingleImageDataset:
    def __init__(self, image_path, transform=None):
        self.image_path = image_path
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(self.image_path).convert('RGB')

        target = {}
        target["file_name"] = [str(self.image_path)]
        target["image_label"] = torch.tensor([0000], dtype=torch.int64)  # Set the target label as needed

        if self.transform is not None:
            img, target = self.transform(img, target)

        return img, target

    def __len__(self):
        return 1  

def make_transforms():
    mean, std = data_mean_std["cub"]

    normalize = T.Compose([
        T.ToTensor(),
        T.Normalize(mean, std)
    ])
    return T.Compose([
        T.RandomResize([800], max_size=1333),
        normalize,
    ])

def build_single_image_dataset(image_path):
    transform = make_transforms()
    dataset = SingleImageDataset(image_path=image_path, transform=transform)
    return dataset

def image_preprocessing(image_path):
    single_image_dataset = build_single_image_dataset(image_path)
    sampler_val = torch.utils.data.SequentialSampler(single_image_dataset)
    data_loader_val = DataLoader(single_image_dataset, args.batch_size, sampler=sampler_val,
                                 drop_last=False, collate_fn=utils.collate_fn, num_workers=1) 
    return data_loader_val

In [5]:
def load_model(args, model):
    checkpoint = torch.load(args.resume, map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    return model

In [6]:
@torch.no_grad()
def predicted_class(output, topk=(1,)):
    query_logits = output['query_logits']
    batch_size = query_logits.size(0)
    
    maxk = min(max(topk), query_logits.size(1))
    _, pred = query_logits.topk(maxk, dim=1, largest=True, sorted=True)
    return pred

In [7]:
def get_class_name(number, filename="demo_image/classes.txt"):
    with open(filename, "r") as file:
        lines = file.readlines()
    for line in lines:
        parts = line.strip().split(" ", 1)
        if parts[0] == str(number):
            class_name = parts[1]
            break
    else:
        return f"No class found for number {number}"

    class_name_parts = class_name.split(".")
    modified_name = " ".join(part.capitalize() for part in class_name_parts[1].split("_"))

    return modified_name


In [8]:
def SuperImposeHeatmap(attention, input_image):
    alpha=0.5
    avg_heatmap_resized = cv2.resize(attention, (input_image.shape[1], input_image.shape[0]), interpolation=cv2.INTER_CUBIC)
    avg_normalized_heatmap = (avg_heatmap_resized - np.min(avg_heatmap_resized)) / (np.max(avg_heatmap_resized) - np.min(avg_heatmap_resized))
    heatmap = (avg_normalized_heatmap * 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.medianBlur(heatmap,15)
    heatmap =  cv2.GaussianBlur(heatmap, (15, 15), 0)
    result = (input_image *alpha  + heatmap * (1-alpha)).astype(np.uint8)
    return result

def get_heatmap(latent_activation, input_image, constant_color_scale=False):
    image_a = latent_activation.cpu().numpy()
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    if constant_color_scale:
        reshaped_image_a = np.concatenate((reshaped_image_a, np.zeros((reshaped_image_a.shape[1], 1)), np.ones((reshaped_image_a.shape[1], 1))*255), axis=1)
    
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)

    if constant_color_scale:
        heatmap_colored = heatmap_colored[:, :-2]
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

In [10]:


import pdb

def visualization(args, image_path, query_index, attention_scores, encoder_output):

    attention_score = attention_scores[5, :, :, :, :] # sixth layer of the decoder
    attention_score_pred_query = attention_score[:, :, query_index, :] # query corresponding to predicted class 

    # Load and resize the original image
    input_image = cv2.imread(image_path)
    input_image = cv2.resize(input_image, (0, 0), fx=0.8, fy=0.8)

    # Create a subplot grid for the original image and visualizations
    num_heads = attention_score_pred_query.shape[1] + 1  
    rows = 1
    cols = num_heads // rows

    # Plot the original image
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axes[0].imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    axes[0].axis('off')

    for head_index in range(attention_score_pred_query.shape[1]):
        heatmap_head = attention_score_pred_query[:, head_index, :].reshape(encoder_output.shape[2], encoder_output.shape[3]).detach().cpu().numpy()
        result = SuperImposeHeatmap(heatmap_head, input_image) # heatmap_head-><class 'numpy.ndarray'>(25, 25) 

        if rows == 1 and cols == 1:
            ax = axes
        elif rows == 1:
            ax = axes[head_index + 1]
        elif cols == 1:
            ax = axes[head_index + 1]
        else:
            ax = axes[(head_index + 1) // cols, (head_index + 1) % cols]

        ax.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        ax.axis('off')

    plt.tight_layout()
    plt.show()

def visualization(args, image_path, query_index, attention_scores, encoder_output):

    attention_score = attention_scores[5, :, :, :, :] # sixth layer of the decoder
    attention_score_pred_query = attention_score[:, :, query_index, :] # query corresponding to predicted class 

    image = T.Resize(size=(args.image_size, args.image_size))(Image.open(image_path))
    img_tensor = T.ToTensor()(image)

    # Load and resize the original image
    input_image = cv2.imread(image_path)
    input_image = cv2.resize(input_image, (0, 0), fx=0.8, fy=0.8)

    # Create a subplot grid for the original image and visualizations
    num_heads = attention_score_pred_query.shape[1] + 1  
    rows = 1
    cols = num_heads // rows

    # Plot the original image
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axes[0].imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    axes[0].axis('off')

    for head_index in range(attention_score_pred_query.shape[1]):
        heatmap_head = attention_score_pred_query[:, head_index, :].reshape(encoder_output.shape[2], encoder_output.shape[3]).detach().cpu().numpy()
        # get_heatmap(latent_activation, input_image
        result = SuperImposeHeatmap(heatmap_head, input_image) # heatmap_head-><class 'numpy.ndarray'>(25, 25) 

        if rows == 1 and cols == 1:
            ax = axes
        elif rows == 1:
            ax = axes[head_index + 1]
        elif cols == 1:
            ax = axes[head_index + 1]
        else:
            ax = axes[(head_index + 1) // cols, (head_index + 1) % cols]

        ax.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [12]:
import pdb

def main(args):

    dataset_val = build_dataset(image_set=args.test, args=args)
    label_to_spcname = {label: classname for classname, label in dataset_val.class_to_idx.items()}
    
    image_path = "/home/harishbabu/data/cub190_imgnet/train/cub_012_Yellow_headed_Blackbird/Yellow_Headed_Blackbird_0017_8511.jpg"
    
    test_image_loader=image_preprocessing(image_path)

    device = torch.device(args.device)
    # model, _ = build_model(args)
    model, criterion, spclabel_to_anclabels= build_model_hierINTR(args, label_to_spcname)
    model.to(device)
    model=load_model(args, model)
    model.eval()

    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'
    for samples, targets in metric_logger.log_every(test_image_loader, 1, header):
        samples = samples.to(device)
    
        outputs, encoder_output, _, attention_scores, avg_attention_scores = model(samples)

        pred_class_index=predicted_class(outputs, topk=(1, 1))
        # print ("Species predicted by INTR is:",get_class_name(int(pred_class_index[0][0])+1), "\n")
        print ("Species predicted by HierINTR is:",label_to_spcname[int(pred_class_index[0][0])], "\n")

        # visualization(args, image_path, int(pred_class_index[0][0]), attention_scores, encoder_output)

if __name__ == '__main__':
    parser = argparse.ArgumentParser('INTR interpretation visualization script', parents=[get_args_parser()])
    args = parser.parse_args(args=[])

    args.phylo_config = 'configs/cub190_phylogeny_disc4.yaml'
    args.resume = 'output_HierINTR/cub190_imgnet/001_cub190-imgnet_disc=4/cub_checkpoint.pth'
    args.dataset_path = "/home/harishbabu/data/"
    args.dataset_name = "cub190_imgnet"
    main(args)

In build function
Built backbone
Built transformer
Built tree
Set anc labels
Species predicted by HierINTR is: cub_012_Yellow_headed_Blackbird 

Test:  [0/1]  eta: 0:00:00    time: 0.3410  data: 0.2466  max mem: 2172
Test: Total time: 0:00:00 (0.4159 s / it)


torch.Size([6, 1, 8, 190, 625])


ipdb>  attention_score = attention_scores[5, :, :, :, :]
ipdb>  attention_score.shape


torch.Size([1, 8, 190, 625])


ipdb>  pred_class_index=predicted_class(outputs, topk=(1, 1))
ipdb>  query_index = int(pred_class_index[0][0])
ipdb>  query_index 


...

ipdb>  head_index = 0
ipdb>  heatmap_head = attention_score_pred_query[:, head_index, :].reshape(encoder_output.shape[2], encoder_output.shape[3]).detach().cpu().numpy()
ipdb>  heatmap_head.shape


(25, 25)


ipdb>  type(heatmap_head)


<class 'numpy.ndarray'>


ipdb>  q


In [17]:
!pwd

/home/harishbabu/projects/INTR


# Knock-out

In [11]:
parser = argparse.ArgumentParser('INTR interpretation visualization script', parents=[get_args_parser()])
args = parser.parse_args(args=[])

args.phylo_config = 'configs/cub190_phylogeny_disc4.yaml'
args.resume = 'output_HierINTR/cub190_imgnet/001_cub190-imgnet_disc=4/cub_checkpoint.pth'
args.dataset_path = "/home/harishbabu/data/"
args.dataset_name = "cub190_imgnet"

dataset_val = build_dataset(image_set=args.test, args=args)
spcname_to_label = dataset_val.class_to_idx
label_to_spcname = {label: classname for classname, label in dataset_val.class_to_idx.items()}

image_path = "/home/harishbabu/data/cub190_imgnet/train/cub_012_Yellow_headed_Blackbird/Yellow_Headed_Blackbird_0017_8511.jpg"

test_image_loader=image_preprocessing(image_path)

device = torch.device(args.device)
# model, _ = build_model(args)
model, criterion, spclabel_to_anclabels= build_model_hierINTR(args, label_to_spcname)
model.to(device)
model=load_model(args, model)
model.eval()

In build function


/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/harishbabu/.conda/envs/hpnet4/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/harishbabu/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 101MB/s] 


Built backbone
Built transformer
Built tree
Set anc labels


HierINTR(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256

In [32]:
spcname_to_label['cub_012_Yellow_headed_Blackbird']

11

In [16]:
# model.spclabel_to_anclabels
# spcname_to_label

def get_lvl_cousins(level, spclabel, spclabel_to_anclabels):
    # return the labels of cousins at a particular level

    assert level < len(list(spclabel_to_anclabels.values())[0])
    species_level_label = spclabel_to_anclabels[spclabel][level]
    return [cousin_spclabel for cousin_spclabel, anclabels in spclabel_to_anclabels.items() \
             if ((anclabels[level] == species_level_label) and (cousin_spclabel != spclabel))]
    
cousin_labels = get_lvl_cousins(level=1, spclabel=11, spclabel_to_anclabels=model.spclabel_to_anclabels)


In [17]:
cousin_labels

[8,
 9,
 10,
 12,
 20,
 24,
 25,
 46,
 72,
 84,
 90,
 91,
 92,
 93,
 105,
 106,
 107,
 108,
 109,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 138]

In [13]:
model.spclabel_to_anclabels

{0: [0, 0, 0],
 1: [0, 0, 0],
 2: [0, 0, 0],
 3: [1, 2, 2],
 4: [2, 4, 5],
 5: [2, 4, 5],
 6: [2, 4, 5],
 7: [2, 4, 6],
 8: [3, 6, 13],
 9: [3, 6, 14],
 10: [3, 6, 13],
 11: [3, 6, 15],
 12: [3, 6, 15],
 13: [3, 7, 26],
 14: [3, 7, 27],
 15: [3, 7, 27],
 16: [3, 8, 29],
 17: [4, 15, 47],
 18: [5, 16, 48],
 19: [3, 9, 31],
 20: [3, 6, 16],
 21: [6, 20, 58],
 22: [6, 20, 58],
 23: [6, 20, 58],
 24: [3, 6, 14],
 25: [3, 6, 14],
 26: [5, 17, 51],
 27: [7, 21, 60],
 28: [7, 21, 61],
 29: [1, 3, 3],
 30: [1, 3, 4],
 31: [1, 3, 3],
 32: [3, 10, 41],
 33: [3, 10, 41],
 34: [8, 24, 67],
 35: [9, 28, 74],
 36: [9, 29, 78],
 37: [9, 28, 75],
 38: [9, 28, 76],
 39: [9, 29, 78],
 40: [9, 28, 77],
 41: [9, 28, 75],
 42: [0, 1, 1],
 43: [10, 30, 79],
 44: [3, 11, 43],
 45: [3, 11, 43],
 46: [3, 6, 13],
 47: [11, 31, 80],
 48: [11, 31, 80],
 49: [11, 32, 81],
 50: [11, 33, 82],
 51: [3, 7, 27],
 52: [3, 12, 44],
 53: [3, 10, 42],
 54: [3, 7, 28],
 55: [2, 4, 7],
 56: [2, 5, 9],
 57: [2, 5, 9],
 58: [2